# Partial Differential Equations (PDEs)

Here we use [MethodOfLines.jl](http://methodoflines.sciml.ai/dev/) to sumbolically define the PDE system in finite difference method (FDM).

## Brusselator PDE

The Brusselator PDE is defined as follows:

$$
\begin{align}
\frac{\partial u}{\partial t} &= 1 + u^2v - 4.4u + \alpha (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}) + f(x, y, t) \\
\frac{\partial v}{\partial t} &= 3.4u - u^2 v + \alpha (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2})
\end{align}
$$

where

$$
f(x, y, t) = 
\begin{cases}
5 \qquad \text{if} (x - 0.3)^2 + (y - 0.6)^2 \leq 0.1^2 \ and \  t \geq 1.1  \\
0 \qquad \text{otherwise}
\end{cases}
$$

and the initial conditions are

$$
\begin{align}
u(x, y, 0) &= 22(y(1-y))^{1.5} \\
v(x, y, 0) &= 27(x(1-x))^{1.5}
\end{align}
$$

with the periodic boundary condition

$$
\begin{align}
u(x+1, y, 0) &= u(x, y, t)  \\
u(x, y+1, 0) &= u(x, y, t)
\end{align}
$$

on a timespan of $t \in [0, 11.5]$.

In [ ]:
using ModelingToolkit, MethodOfLines, DifferentialEquations, DomainSets

### PDE system

In [ ]:
@parameters x y t
@variables u(..) v(..)

Dt = Differential(t)
Dx = Differential(x)
Dy = Differential(y)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

# Laplacian
∇²(u) = Dxx(u) + Dyy(u)

In [ ]:
brusselator_f(x, y, t) = (((x-0.3)^2 + (y-0.6)^2) <= 0.1^2) * (t >= 1.1) * 5

In [ ]:
x_min = y_min = t_min = 0.0
x_max = y_max = 1.0
t_max = 11.5
α = 10.0

In [ ]:
u0(x,y,t) = 22(y*(1-y))^(3/2)
v0(x,y,t) = 27(x*(1-x))^(3/2)

In [ ]:
eq = [Dt(u(x,y,t)) ~ 1. + v(x,y,t)*u(x,y,t)^2 - 4.4*u(x,y,t) + α*∇²(u(x,y,t)) + brusselator_f(x, y, t),
      Dt(v(x,y,t)) ~ 3.4*u(x,y,t) - v(x,y,t)*u(x,y,t)^2 + α*∇²(v(x,y,t))]

In [ ]:
domains = [x ∈ Interval(x_min, x_max),
           y ∈ Interval(y_min, y_max),
           t ∈ Interval(t_min, t_max)]

In [ ]:
# Periodic BCs
bcs = [u(x,y,0) ~ u0(x,y,0),
       u(0,y,t) ~ u(1,y,t),
       u(x,0,t) ~ u(x,1,t),

       v(x,y,0) ~ v0(x,y,0),
       v(0,y,t) ~ v(1,y,t),
       v(x,0,t) ~ v(x,1,t)] 

In [ ]:
@named pdesys = PDESystem(eq, bcs, domains, [x,y,t], [u(x,y,t),v(x,y,t)])

### Discretization

In [ ]:
N = 32

dx = (x_max-x_min)/N
dy = (y_max-y_min)/N

order = 2

discretization = MOLFiniteDifference([x=>dx, y=>dy], t, approx_order=order, grid_align=center_align)

In [ ]:
# Convert the PDESystem in to an ODEProblem or NonlinearProblem.
prob = discretize(pdesys,discretization)

In [ ]:
sol = solve(prob, TRBDF2(), saveat=0.1)

### Extracting results

In [ ]:
grid = get_discrete(pdesys, discretization)
discrete_x = grid[x]
discrete_y = grid[y]
discrete_t = sol[t]

solu = [map(d -> sol[d][i], grid[u(x, y, t)]) for i in 1:length(sol[t])]
solv = [map(d -> sol[d][i], grid[v(x, y, t)]) for i in 1:length(sol[t])]

In [ ]:
using Plots

anim = @animate for k in 1:length(discrete_t)
    heatmap(solu[k][2:end, 2:end], title="u @ t=$(discrete_t[k])") # 2:end since end = 1, periodic condition
end

mp4(anim, "../_static/Brusselator2Dsol_u.mp4", fps = 8)

In [ ]:
anim = @animate for k in 1:length(discrete_t)
    heatmap(solv[k][2:end, 2:end], title="v @ t=$(discrete_t[k])")
end

mp4(anim, "../_static/Brusselator2Dsol_v.mp4", fps = 8)